In [1]:
#TDA API testing

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import math
import datetime

In [3]:
import env

Polygon.io API key loaded up.
Cboe API key loaded up.
TD Ameritrade API key loaded up.
Coinbase API key loaded up.
All credentials loaded successfully


In [4]:
import tda

In [5]:
tda_key = env.tda_api_key
tda_url = env.my_redirect_url

# tda_url

In [6]:
tda

<module 'tda' from '/usr/local/anaconda3/lib/python3.8/site-packages/tda/__init__.py'>

In [7]:
import os

In [8]:
import atexit
import datetime
import dateutil
import httpx
import sys
import tda

API_KEY = env.tda_api_key
REDIRECT_URI = env.my_redirect_url
TOKEN_PATH = 'ameritrade-token.json'
YOUR_BIRTHDAY = datetime.datetime(year=1986, month=2, day=23)
SP500_URL = "https://tda-api.readthedocs.io/en/latest/_static/sp500.txt"

def make_webdriver():
# Import selenium here because it's slow to import from selenium import webdriver
    from selenium import webdriver
    
    driver = webdriver.Chrome()
    atexit.register(lambda: driver.quit())
    return driver

# Create a new client
client = tda.auth.easy_client(
    API_KEY,
    REDIRECT_URI,
    TOKEN_PATH,
    make_webdriver)


# Load S&P 500 composition from documentation
# sp500 = httpx.get(
#     SP500_URL, headers={
#         "User-Agent": "Mozilla/5.0"}).read().decode().split()

# Fetch fundamentals for all symbols and filter out the ones with ex-dividend
# dates in the future and dividend payment dates on your birth month. Note we
# perform the fetch in two calls because the API places an upper limit on the
# number of symbols you can fetch at once.
# today = datetime.datetime.today()
# birth_month_dividends = []
# for s in (sp500[:250], sp500[250:]):
#     r = client.search_instruments(
#         s, tda.client.Client.Instrument.Projection.FUNDAMENTAL)
#     assert r.status_code == 200, r.raise_for_status()

#     for symbol, f in r.json().items():

#         # Parse ex-dividend date
#         ex_div_string = f['fundamental']['dividendDate']
#         if not ex_div_string.strip():
#             continue
#         ex_dividend_date = dateutil.parser.parse(ex_div_string)

#         # Parse payment date
#         pay_date_string = f['fundamental']['dividendPayDate']
#         if not pay_date_string.strip():
#             continue
#         pay_date = dateutil.parser.parse(pay_date_string)

#         # Check dates
#         if (ex_dividend_date > today
#                 and pay_date.month == YOUR_BIRTHDAY.month):
#             birth_month_dividends.append(symbol)

# if not birth_month_dividends:
#     print('Sorry, no stocks are paying out in your birth month yet. This is ',
#           'most likely because the dividends haven\'t been announced yet. ',
#           'Try again closer to your birthday.')
#     sys.exit(1)

# Purchase one share of each the stocks that pay in your birthday month.
# account_id = int(input(
#     'Input your TDA account number to place orders (<Ctrl-C> to quit): '))
# for symbol in birth_month_dividends:
#     print('Buying one share of', symbol)

    # Build the order spec and place the order
#     order = tda.orders.equities.equity_buy_market(symbol, 1)

#     r = client.place_order(account_id, order)
#     assert r.ok, r.raise_for_status()

In [9]:
client.get_option_chain("AAPL")

<Response [200 OK]>

In [10]:
# Trying to get historical data into a dataframe:

response = client.get_price_history("AAPL").content

In [ ]:
# only print this if I actually need to see it. It fills up the GitHub page.

# response

In [12]:
import requests

In [13]:
td_consumer_key = env.tda_api_key

In [14]:
import json

In [15]:
# response = requests.get('https://api.tdameritrade.com/v1/marketdata/AAPL/pricehistory?periodType=month&frequencyType=daily&startDate=1464148800000&endDate=1464825600000')
response = requests.get('https://api.tdameritrade.com/v1/marketdata/{AAPL}/quotes')

In [16]:
endpoint = 'https://api.tdameritrade.com/v1/marketdata/{stock_ticker}/quotes'

In [17]:
full_url = endpoint.format(stock_ticker = 'AAPL')

page = requests.get(url = full_url, params = {'apikey' : td_consumer_key})

content = json.loads(page.content)

In [18]:
# Creating a function to return stock price:

# def stock_price(url = full_url, params = {'apikey' : td_consumer_key}):
    

In [19]:
# This should be the GET command I need:

# GET /v1/marketdata/chains?apikey='myAUTHENTICATIONcodegoeshere'&symbol=AAPL&contractType=CALL&strikeCount=5&strategy=SINGLE&toDate=2021-06-18 HTTP/1.1

# And tie it together with the resource URL:
# https://api.tdameritrade.com/v1/marketdata/chains

In [20]:
requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=5&strategy=SINGLE&toDate=2021-06-18 HTTP/1.1')

<Response [200]>

In [21]:
aapl_api_test = requests.get(f'https://api.tdameritrade.com//v1/marketdata/chains?apikey={tda_key}&symbol=AAPL&contractType=CALL&strikeCount=5&strategy=SINGLE&toDate=2021-06-18 HTTP/1.1')

In [22]:
aapl_content = aapl_api_test.content

In [ ]:
aapl_content

In [24]:
num = 7

In [25]:
factorial = 1

In [26]:
if num < 0:
    print("Sorry, factorial does not exist for negative numbers")
elif num == 0:
    print("The factorial of 0 is 1")
else:
    for i in range(1, num + 1):
        factorial = factorial * i
    print("The factorial of", num, 'is', factorial)

The factorial of 7 is 5040
